<a href="https://colab.research.google.com/github/alisonsoong/NASA-SEES-Internship-2021/blob/main/WildfireWorkUpdate7_21_2021_FirstTry.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Created by Alison Soong


# Notes

Risk factors:
- High temperatures (extreme heat)
- Low humidity

Algorithm ideas:
- https://www.ci.richmond.ca.us/DocumentCenter/View/53610/City-of-Richmond-Wildfire-Preparedness-and-Evacuation-Guide "A home within 10 miles of a wildfire will most likely be effected by wind driven embers which can be a risk to your property" --> might be a good buffer area to look at


Datasets:
- https://developers.google.com/earth-engine/datasets/catalog/WorldPop_GP_100m_pop#bands
- https://developers.google.com/earth-engine/datasets/catalog/MODIS_006_MCD64A1
- https://developers.google.com/earth-engine/datasets/catalog/IDAHO_EPSCOR_GRIDMET#bands (humidity, temperature, wind)

References:
- https://colab.research.google.com/github/giswqs/geemap/blob/master/examples/notebooks/00_geemap_key_features.ipynb#scrollTo=zduDFJXjyeiz
- 


# Import libraries

In [1]:
import ee # Import Earth Engine API

In [2]:
## Trigger the authentication flow. You only need to do this once
ee.Authenticate()

# Initialize the library.
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=i7904W-dxz-m72-_84by0UU2flqtw8cN7bK1UHaoCio&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AX4XfWiEWh34CxLyDDTov13nnI4Lo9wDXqpJPbJQ9dkhJB-mimZGWAs_m5Y

Successfully saved authorization token.


In [3]:
# Installs geemap package
import subprocess

try:
    import geemap
    print("geemap is imported and ready to use in Colab")
except ImportError:
    print('geemap package not installed. Installing ...')
    subprocess.check_call(["python", '-m', 'pip', 'install', 'geemap'])
    import geemap
    print("geemap is now installed, imported and ready to use in Colab") 


geemap package not installed. Installing ...
geemap is now installed, imported and ready to use in Colab


In [4]:
try:
    import geemap
    import ee
    import seaborn as sns
    import matplotlib.pyplot as plt
except ModuleNotFoundError:
    if 'google.colab' in str(get_ipython()):
        print("package not found, installing w/ pip in Google Colab...")
        !pip install geemap seaborn matplotlib
    else:
        print("package not found, installing w/ conda...")
        !conda install mamba -c conda-forge -y
        !mamba install geemap -c conda-forge -y
        !conda install seaborn matplotlib -y
    import geemap
    import ee
    import seaborn as sns
    import matplotlib.pyplot as plt

# Other ideas

In [ ]:
# Map.addLayer(population2, pop_params, "Population 2020 masked")
# dates = np.arange(2000, 2020)
# for year in dates:
#     date1 = str(year)+'-01-01'
#     date2 = str(year+1)+'-01-01'
#     burned = ee.ImageCollection('MODIS/006/MCD64A1').filter(ee.Filter.date('2020-01-01', '2021-01-01'))
#     popdataset = ee.ImageCollection("WorldPop/GP/100m/pop")\
#       .filterDate(date1, date2)
#     population = popdataset.mean().clip(california_area)
#     burnedImage = burned.select('BurnDate').mean().clip(california_area)
#     title = "population " + str(year)
#     title2 = "burned " + str(year)
#     Map.addLayer(population, pop_params, title, False)
#     Map.addLayer(burnedImage, burned_params2, title2, False, 0.3)

# burnedBuffer = burned.reduceToVectors().reproject(proj);

# burnedBuffer.getInfo()

# population2 = popdataset2020.mean().clip(burnedBuffer)

# buffer around areas 
# classes = burned.reduceToVectors(
#   reducer=ee.Reducer.countEvery(), 
#   geometry=california_area,
#   scale= 30,
#   maxPixels=1e9
# )

# result = ee.FeatureCollection(classes);

# burnedBuffer = burned2020.select('BurnDate').mean().clip(california_area).geometry()


In [ ]:
# Land cover classification using Random Forest (Doesn't quite work)

# Part 1: Adding imagery, filtering to area and date range, masking out clouds, and making a composite

image = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR').filterBounds(california_area) # to filter the image collection to an area

# Function to cloud mask from the pixel_qa band of Landsat 8 SR data. Bits 3 and 5 are cloud shadow and cloud, respectively.
def maskL8sr(image):
    cloudShadowBitMask = 1 << 3 # bitwise shift left, selecting pixels that have value 3 (cloud shadow)
    cloudsBitMask = 1 << 5 # bitwise shift left, selecting pixels that are labeled 5 (clouds)

    qa = image.select('pixel_qa')

    mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0).And(qa.bitwiseAnd(cloudsBitMask).eq(0))

    return image.updateMask(mask).divide(10000).select("B[0-9]*").copyProperties(image, ["system:time_start"])


# Filter imagery for 2019 and 2020 summer date ranges. 
# Create joint filter and apply it to Image Collection.
sum20 = ee.Filter.date('2020-06-01','2020-09-30') # summer 2020 data, with date range
sum19 = ee.Filter.date('2019-06-01','2019-09-30') # summer 2019 data, with date range

SumFilter = ee.Filter.Or(sum20, sum19) # using both two date ranges in the filter (so a combination of them)

allsum = image.filter(SumFilter) # taking that image and filtering it to the summer date ranges

# Make a Composite: Apply the cloud mask function, use the median reducer, and clip the composite to our area of interest
composite = allsum.map(maskL8sr).median().clip(california_area)

#Display the Composite
# Map.addLayer(composite, {bands: ['B4','B3','B2'],min: 0, max: 0.3},'California Color Image', 0);

# Part 2: Add Developed Land Data******

# //Add the impervious surface layer
# latest data set that we can get! The best thing that we can do~
# the most recent and best available data that we can use
def func(image):
  return image.clip(california_area)
impervious = ee.ImageCollection('USGS/NLCD').filterDate('2016-01-01', '2017-01-01').filterBounds(california_area).select('impervious').map(func)

# //Reduce the image collection to 
reduced = impervious.reduce('median') # reduce to a single image

# //Mask out the zero values in the data
masked = reduced.selfMask() # masking out the zero values in the data. taking out any values in the data that are zero (not interested in them)

# ////******Part 3: Prepare for the Random Forest model******
# ////////////////////////////////////////////////

# //// In this example, we use land cover classes: 
# //// 1-100 = Percent Impervious Surfaces
# //// 101 = coniferous  
# //// 102 = mixed forest
# //// 103 = deciduous
# //// 104 = cultivated
# //// 105 = water
# //// 106 = cloud

# Merge land cover classifications into one feature class
newfc = coniferous.merge(mixedforest).merge(deciduous).merge(cultivated).merge(water) # merge to include all these different cover classifications. still saving/maintaining their original values though

# Specify the bands to use in the prediction.
bands = ['B3', 'B4', 'B5', 'B6', 'B7']  # for the random forest classification 

# Make training data by 'overlaying' the points on the image.
points = composite.select(bands).sampleRegions({'collection': newfc,'properties': ['landcover'], 'scale': 30}).randomColumn() 

# //Randomly split the samples to set some aside for testing the model's accuracy
# //using the "random" column. Roughly 80% for training, 20% for testing.
split = 0.8 # randomly splitting samples, 80 for training, 20 percent for testing
training = points.filter(ee.Filter.lt('random', split));
testing = points.filter(ee.Filter.gte('random', split));

# //Print these variables to see how much training and testing data you are using
print('Samples n =', points.aggregate_count('.all'));
print('Training n =', training.aggregate_count('.all'));
print('Testing n =', testing.aggregate_count('.all'));

# //******Part 4: Random Forest Classification and Accuracy Assessments******
# //////////////////////////////////////////////////////////////////////////

# //Run the RF model using 300 trees and 5 randomly selected predictors per split ("(300,5)"). 
# //Train using bands and land cover property and pull the land cover property from classes
# // Random Forest Model: Random Forest is a tree-based machine learning algorithm that uses a series of
# // decision trees to select the best classification for all pixels within imagery.

# var classifier = ee.Classifier.smileRandomForest(300,5).train({ //  random forest model (300 trees, 5 selected predictivors per split)
# features: training, // the training data from above (the points, the approximately 800 points, or 80 percent of alld ata points)
# classProperty: 'landcover', // what we are looking to classify for
# inputProperties: bands // the bands to be used in the prediction
# });

# //Test the accuracy of the model

# //Print Confusion Matrix and Overall Accuracy
# var confusionMatrix = classifier.confusionMatrix(); // classifier is the random forest model from above. SANITY CHECK. checking against data it was trained on.
# print('Confusion matrix: ', confusionMatrix); // the 80 percent of data used to train classifier. confusion matrix: will compute for the random classifer based on the training data. the sanity check
# print('Training Overall Accuracy: ', confusionMatrix.accuracy());  // the accuracy of the confusion matrix. returns floating point number that indicates the accuracy of the confusion matrix. number of correct classifications divided by total classifications
# var kappa = confusionMatrix.kappa(); // kappa accuracy, and agreement between classifications and reference data (how well the classifications are doing versus randomly assigning values)
# print('Training Kappa', kappa);
 
# var validation = testing.classify(classifier); // now looking at the agreement between the classifier we trained and the completely new data for the classifier, and checking how it matches up! better check of how accurate classifier is for new data (the withheld data)
# var testAccuracy = validation.errorMatrix('landcover', 'classification'); // the 20 percent of data withheld from training
# print('Validation Error Matrix RF: ', testAccuracy);
# print('Validation Overall Accuracy RF: ', testAccuracy.accuracy());
# var kappa1 = testAccuracy.kappa();
# print('Validation Kappa', kappa1);

# //Apply the trained classifier to the image
# var classified = composite.select(bands).classify(classifier); // running the model of our forest classification!

# ////******Part 5:Create a legend******

# //Set position of panel

# legend = ui.Panel({
#   style: {
#     position: 'bottom-left',
#     padding: '8px 15px'
#   }
# });
 
# //Create legend title
# var legendTitle = ui.Label({
#   value: 'Classification Legend', // the title of the legend
#   style: {
#     fontWeight: 'bold',
#     fontSize: '18px',
#     margin: '0 0 4px 0',
#     padding: '0'
#     }
# });
 
# //Add the title to the panel
# legend.add(legendTitle);
 
# //Create and style 1 row of the legend.
# var makeRow = function(color, name) {
 
#       var colorBox = ui.Label({
#         style: {
#           backgroundColor: '#' + color,
#           padding: '8px',
#           margin: '0 0 4px 0'
#         }
#       });
      
#       var description = ui.Label({
#         value: name,
#         style: {margin: '0 0 4px 6px'}
#       });
 
#       return ui.Panel({
#         widgets: [colorBox, description],
#         layout: ui.Panel.Layout.Flow('horizontal')
#       });
# };
 

legend_dict = {
    'Low Density Development': 'CCADE0',
    'Mid Density Development':'A052D3',
    'High Density Development': '633581',
    'Coniferous': '18620f',
    'Mixed Forest': '3B953B',
    'Deciduous': '89CD89',
    'Cultivated': 'EFE028',
    'Water': '0b4a8b'
}

# landcover = ee.Image('USGS/NLCD/NLCD2016').select('landcover')

# //Identify palette with the legend colors
# var palette =['CCADE0', 'A052D3', '633581', '18620f', '3B953B','89CD89', 'EFE028', '0b4a8b'];
 
# //Identify names within the legend
# var names = ['Low Density Development','Mid Density Development','High Density Development',
#             'Coniferous','Mixed Forest','Deciduous','Cultivated','Water'];
 
# //Add color and names
# for (var i = 0; i < 8; i++) {
#   legend.add(makeRow(palette[i], names[i]));
#   }  

# //Add legend to map
# Map.add(legend);

# ////******Part 6: Display the Final Land Cover Classification and Provide Export Options******
# //////////////////////////////////////////////////////////////////////////////////////////////

# //Create palette for the final land cover map classifications // the 'quantity' values are just the values assigned to each category/classification!
urbanPalette =  '<RasterSymbolizer>' + ' <ColorMap  type="intervals">' + '<ColorMapEntry color="#CCADE0" quantity="22" label="Low Density Development"/>' + '<ColorMapEntry color="#A052D3" quantity="56" label="Mid Density Development"/>' + '<ColorMapEntry color="#633581" quantity="100" label="High Density Development"/>' + '<ColorMapEntry color="#18620f" quantity="101" label="Coniferous"/>' + '<ColorMapEntry color="#3B953B" quantity="102" label="Mixed Forest"/>' + '<ColorMapEntry color="#89CD89" quantity="103" label="Deciduous"/>' + '<ColorMapEntry color="#EFE028" quantity="104" label="Cultivated"/>' + '<ColorMapEntry color="#0b4a8b" quantity="105" label="Water"/>' + '</ColorMap>' + '</RasterSymbolizer>'

# //Mask out impervious surfaces
finalmap = classified.blend(masked) # classified = randomm forest classifciation, masked = impervious surface layer (percent impervious)



# Looking at populations and burn scar (burn areas)

In [44]:
california_area = ee.FeatureCollection("TIGER/2018/States").filter(ee.Filter.eq('NAME', 'California'))

def mask(image):
  empty = 0
  qa = image.select('population');
  mask = qa.equal(0) 
  return image.updateMask(mask).select("population").copyProperties(image, ["system:time_start"])

# population grid
popdataset2020 = ee.ImageCollection("WorldPop/GP/100m/pop")\
    .filterDate("2020-01-01", "2021-01-01")

pop2020 = popdataset2020.mean().clip(california_area)

datamask = pop2020
qa = datamask.select('population') 
mask = qa.neq(0)
pop2020masked = pop2020.updateMask(mask.unmask()).clip(california_area)

# 2019
popdataset2019 = ee.ImageCollection("WorldPop/GP/100m/pop")\
    .filterDate("2001-01-01", "2002-01-01")

def func(image):
  return image.clip(burned2020.geometry())
popdataset2019.map(func)

pop2019 = popdataset2019.mean().clip(california_area)

datamask = pop2019
qa = datamask.select('population') 
mask = qa.neq(0)
pop2019masked = pop2019.updateMask(mask.unmask()).clip(california_area)

pop_params = {
  'bands': ['population'],
  'min': 0.0,
  'max': 60.0,
  # 'palette': ['black', '1fff4f', 'd4ff50']
  'palette' : ['black','bdd7e7','6baed6','2171b5']
  # 'palette': ['white', 'green', 'purple']
}

pop_params2 = {
  'bands': ['population'],
  'min': 0.0,
  'max': 60.0,
  # 'palette': ['black', '1fff4f', 'd4ff50']
  'palette' : ['eff3ff','bdd7e7','6baed6','2171b5']
  # 'palette': ['white', 'green', 'purple']
}

# burned
burnedCollection2020 = ee.ImageCollection('MODIS/006/MCD64A1').filter(ee.Filter.date('2020-01-01', '2021-01-01'))

burned2020 = burnedCollection2020.select('BurnDate').mean().clip(california_area)

# 2019
burnedCollection2019 = ee.ImageCollection('MODIS/006/MCD64A1').filter(ee.Filter.date('2001-01-01', '2002-01-01'))

burned2019 = burnedCollection2019.select('BurnDate').mean().clip(california_area)

burned_params = {
  'min': 30.0,
  'max': 341.0,
  # 'palette': ['4e0400', '951003', 'c61503', 'ff1901'],
  'palette':['orange', 'red']
}

roi = ee.Geometry.Point(-122.438005, 37.729844) # (long, lat)


In [45]:
def poi_mean(img):
    mean = img.reduceRegion(reducer=ee.Reducer.mean(), geometry=california_area, scale=30).get('population')
    return img.set('date', img.date().format()).set('mean',mean)

# reduced_img = popdataset2020.map(poi_mean)
# sum = popdataset2019.reduce(ee.Reducer.sum())

pop = ee.ImageCollection("WorldPop/GP/100m/pop")\
      .filterDate("2001-01-01", "2021-01-01")
pop.getInfo()
# print(popdataset2019.aggregate_sum(()))
nested_list = reduced_img.reduceColumns(ee.Reducer.toList(2), ['date','mean']).values().get(0)



In [46]:
Map = geemap.Map()
Map.add_basemap("SATELLITE")
Map.addLayer(pop2020, pop_params, "Population 2020", True)
Map.addLayer(pop2020masked, pop_params2, "Population 2020 masked", False)
Map.addLayer(burned2020, burned_params, 'Burned Area 2020', True, 0.6)
Map.addLayer(pop2019, pop_params, "Population 2001", True)
Map.addLayer(pop2019masked, pop_params2, "Population 2001 masked", False)
Map.addLayer(burned2019, burned_params, 'Burned Area 2001', True, 0.6)
roi = ee.Geometry.Point(-122.438005, 37.729844) # (long, lat)
Map.centerObject(roi, 6)
Map.addLayerControl()
Map

# Narrative

Over time, more and more of the California population is directly affected by wildfires. This led us to wonder how to determine what areas and communities are most at risk, especially with population movement into areas surrounded by more vegetation.

# Wildfire Risk and populations

In [5]:
california_area = ee.FeatureCollection("TIGER/2018/States").filter(ee.Filter.eq('NAME', 'California'))

In [122]:
# Landsat EVI
imageCollection = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR').filterBounds(california_area)

def maskL8sr(imageCollection):
  cloudShadowBitMask = 1 << 3
  cloudsBitMask = 1 << 5

  qa = imageCollection.select('pixel_qa')

  mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0) \
        .And(qa.bitwiseAnd(cloudsBitMask).eq(0))

  return imageCollection.updateMask(mask).divide(10000) \
      .select("B[0-9]*") \
      .copyProperties(imageCollection, ["system:time_start"])
      
stepList = ee.List.sequence(2014,2020)

def process_collection(year):
  startDate = ee.Date.fromYMD(year,5,1)
  endDate = ee.Date.fromYMD(year,9,15)
  composite_i = imageCollection.filterDate(startDate, endDate) \
                        .map(maskL8sr) \
                        .median() \
                        .set('system:time_start',startDate)
  return composite_i
filterCollection = stepList.map(process_collection)

yearlyComposites = ee.ImageCollection(filterCollection)
# print(yearlyComposites, 'Masked and Filtered Composites')

# Add Enhanced Vegetation Index to a function and apply it.
# EVI = 2.5 * ((NIR - Red) / (NIR + 6 * Red – 7.5 * Blue + 1))
def evi(img):
  eviImg = img.select(['B5','B4','B2'],['nir','red','blue'])
  eviImg = eviImg.expression(
      '(2.5 * ((NIR - RED)) / (NIR + 6 * RED - 7.5 * BLUE + 1))',
      {
        'NIR': eviImg.select('nir'),
        'RED': eviImg.select('red'),
        'BLUE': eviImg.select('blue')
      }).rename('EVI')
  return img.addBands(eviImg)

yearlyComposites = yearlyComposites.map(evi)

# print(yearlyComposites, 'With EVI as Band')

# Create image collection of yearly composites, selecting the EVI band.
eviCollection = yearlyComposites.select('EVI')

# Create variables for 2020 yearly composite.
y2020 = eviCollection.filterDate('2019-01-01','2019-12-31') \
  .first() \
  .clip(california_area)

eviParams = {'min': 0, 'max': 1, 'palette': ['white', 'green']}


In [7]:
# Population
popdataset2020 = ee.ImageCollection("WorldPop/GP/100m/pop")\
    .filterDate("2020-01-01", "2021-01-01")

pop2020 = popdataset2020.mean().clip(california_area)

pop_params = {
  'bands': ['population'],
  'min': 0.0,
  'max': 60.0,
  # 'palette': ['black', '1fff4f', 'd4ff50']
  'palette' : ['black','bdd7e7','6baed6','2171b5']
  # 'palette': ['white', 'green', 'purple']
}



In [8]:
legend_dict = {
    '11 Open Water': '466b9f',
    '12 Perennial Ice/Snow': 'd1def8',
    '21 Developed, Open Space': 'dec5c5',
    '22 Developed, Low Intensity': 'd99282',
    '23 Developed, Medium Intensity': 'eb0000',
    '24 Developed High Intensity': 'ab0000',
    '31 Barren Land (Rock/Sand/Clay)': 'b3ac9f',
    '41 Deciduous Forest': '68ab5f',
    '42 Evergreen Forest': '1c5f2c',
    '43 Mixed Forest': 'b5c58f',
    '51 Dwarf Scrub': 'af963c',
    '52 Shrub/Scrub': 'ccb879',
    '71 Grassland/Herbaceous': 'dfdfc2',
    '72 Sedge/Herbaceous': 'd1d182',
    '73 Lichens': 'a3cc51',
    '74 Moss': '82ba9e',
    '81 Pasture/Hay': 'dcd939',
    '82 Cultivated Crops': 'ab6c28',
    '90 Woody Wetlands': 'b8d9eb',
    '95 Emergent Herbaceous Wetlands': '6c9fb8'
}

# most recent data available
landcover = ee.ImageCollection('USGS/NLCD').select('landcover').filterDate("2016-01-01", "2017-01-01").first().clip(california_area)

In [9]:
# Impervious land: most recent data available
impervious = ee.ImageCollection('USGS/NLCD').select('impervious').filterDate("2016-01-01", "2017-01-01").first().clip(california_area)

datamask = impervious
qa = impervious.select('impervious') 
mask = qa.neq(0)
imperviousMasked = impervious.updateMask(mask.unmask()).clip(california_area)

In [42]:
# burned
burnedCollection2020 = ee.ImageCollection('MODIS/006/MCD64A1').filter(ee.Filter.date('2020-01-01', '2021-01-01'))

burned2020 = burnedCollection2020.select('BurnDate').mean().clip(california_area)

burned_params = {
  'min': 30.0,
  'max': 341.0,
  # 'palette': ['4e0400', '951003', 'c61503', 'ff1901'],
  'palette':['orange', 'red']
}

In [142]:
# Risk calculationns

risk_params = {
    'min': 0.0,
    'max': 7.0,
    'palette': ['fff7ec','fee8c8','fdd49e','fdbb84','fc8d59','ef6548','d7301f','990000']
}

# Impervious land scoring!
imperviousCur = impervious.select('impervious')
imperviousZero = imperviousCur.updateMask(imperviousCur.gt(75)) # most development, least risk
imperviousOne = imperviousCur.updateMask(imperviousCur.gt(0).And(imperviousCur.lte(75))) # medium development, medium risk

# Impervious reducing values to intended risk score
imperviousZero = imperviousZero.where(imperviousCur.gt(75), 0) # for example, pixels with values over 75 will be reassigned to 0. Because oof masking above, everything else will be 0.
imperviousOne = imperviousOne.where(imperviousCur.gt(0).And(imperviousCur.lte(75)), 1) # for example, pixels with values under 75 will be reassigned to 1. Because oof masking above, everything else will be 0.

imperviousCur = imperviousCur.where(imperviousCur.gt(75), 0)
imperviousCur = imperviousCur.where(imperviousCur.gt(0).And(imperviousCur.lte(75)), 1)
imperviousCur = imperviousCur.updateMask(landcover.neq(11).And(landcover.neq(12)).And(landcover.neq(90)).And(landcover.neq(95)))

# EVI land scoring!
eviCur = y2020.select('EVI')
eviThree = eviCur.updateMask(eviCur.gt(0.50)) # most vegetation, highest risk
eviTwo = eviCur.updateMask(eviCur.gt(0.15).And(eviCur.lte(0.50))) # medium vegetation, medium risk
eviOne = eviCur.updateMask(eviCur.gt(0.05).And(eviCur.lte(0.15))) # low vegetation, low risk
eviZero = eviCur.updateMask(eviCur.gte(0).And(eviCur.lte(0.05))) # no vegetation, no risk

eviThree = eviThree.where(eviCur.gt(0.50), 3)
eviTwo = eviTwo.where(eviCur.gt(0.15).And(eviCur.lte(0.50)), 2)
eviOne = eviOne.where(eviCur.gt(0.05).And(eviCur.lte(0.15)), 1)
eviZero = eviZero.where(eviCur.gte(0).And(eviCur.lte(0.05)), 0)

# compiling evi them
eviCur = eviCur.where(eviCur.gt(0.50), 3)
eviCur = eviCur.where(eviCur.gt(0.15).And(eviCur.lte(0.50)), 2)
eviCur = eviCur.where(eviCur.gt(0.05).And(eviCur.lte(0.15)), 1)
eviCur = eviCur.where(eviCur.gte(0).And(eviCur.lte(0.05)), 0)
eviCur = eviCur.updateMask(landcover.neq(11).And(landcover.neq(12)).And(landcover.neq(90)).And(landcover.neq(95)))

# Land cover scoring!
lcCur = landcover.select('landcover')
lcZero = lcCur.updateMask(lcCur.eq(11).Or(lcCur.eq(12)).Or(lcCur.eq(23)).Or(lcCur.eq(24)).Or(lcCur.eq(31)).Or(lcCur.eq(90)).Or(lcCur.eq(95)))
lcOne = lcCur.updateMask(lcCur.eq(21).Or(lcCur.eq(22)))
lcTwo = lcCur.updateMask(lcCur.neq(11).And(lcCur.neq(12)).And(lcCur.neq(23)).And(lcCur.neq(24)).And(lcCur.neq(31)).And(lcCur.neq(21)).And(lcCur.neq(22)).And(lcCur.neq(90)).And(lcCur.neq(95)))

lcZero = lcZero.where(lcCur.eq(11).Or(lcCur.eq(12)).Or(lcCur.eq(23)).Or(lcCur.eq(24)).Or(lcCur.eq(31)).Or(lcCur.eq(90)).Or(lcCur.eq(95)), 0)
lcOne = lcOne.where(lcCur.eq(21).Or(lcCur.eq(22)), 1)
lcTwo = lcTwo.where(lcCur.neq(11).And(lcCur.neq(12)).And(lcCur.neq(23)).And(lcCur.neq(24)).And(lcCur.neq(31)).And(lcCur.neq(21)).And(lcCur.neq(22)).And(lcCur.neq(90)).And(lcCur.neq(95)), 2)

lcCur = lcCur.where(lcCur.eq(11).Or(lcCur.eq(12)).Or(lcCur.eq(23)).Or(lcCur.eq(24)).Or(lcCur.eq(31)).Or(lcCur.eq(90)).Or(lcCur.eq(95)), 0)
lcCur = lcCur.where(lcCur.eq(21).Or(lcCur.eq(22)), 1)
lcCur = lcCur.where(lcCur.neq(11).And(lcCur.neq(12)).And(lcCur.neq(23)).And(lcCur.neq(24)).And(lcCur.neq(31)).And(lcCur.neq(21)).And(lcCur.neq(22)).And(lcCur.neq(90)).And(lcCur.neq(95)), 2)

lcCur = lcCur.updateMask(landcover.neq(11).And(landcover.neq(12)).And(landcover.neq(90)).And(landcover.neq(95)))

riskImg = ee.Image().clip(california_area)
# riskImg.addBands('RISK')

# Doesn't work yet :)
riskImg = riskImg.select('RISK')
def risk(image):
  riskImg = image
  riskImg = riskImg.expression(
      'zeroEVI + oneEVI + twoEVI*2 + threeEVI*3 + zeroImpervious + oneImpervious + zeroLC + oneLC + twoLC*2',
      {
        'zeroEVI': eviZero.select('EVI'),
        'oneEVI': eviOne.select('EVI'),
        'twoEVI': eviTwo.select('EVI'),
        'threeEVI': eviThree.select('EVI'),
        'zeroImpervious': imperviousZero.select('impervious'),
        'oneImpervious': imperviousOne.select('impervious'),
        # 'zeroLC': lcZero.select('landcover'),
        # 'oneLC': lcOne.select('landcover'),
        # 'twoLC': lcTwo.select('landcover')
      }).rename('RISK')
  return riskImg


def risk2(image, evi, impervious, lc):
  riskImg = image
  eviCur = evi
  imperviousCur = impervious
  lcCur = lc
  riskImg = riskImg.expression(
      'evi+lc',
      {
      'evi': eviCur.select('EVI'),
      'lc': lcCur.select('landcover'),
      # 'impervious': imperviousCur.select('impervious')
      }).rename('RISK')
  return riskImg

# https://developers.google.com/earth-engine/apidocs/ee-image-addbands

riskImg = risk2(riskImg, eviCur, imperviousCur, lcCur)
riskImg = riskImg.updateMask(riskImg.neq(0))
riskImmg = riskImg.updateMask(landcover.select('landcover').eq(11))
risImg = riskImg.select('RISK')


In [143]:
# Map it!

Map = geemap.Map()

Map.add_basemap("SATELLITE")

# EVI
Map.addLayer(y2020, eviParams, '2020 EVI', False)

# Population
Map.addLayer(pop2020, pop_params, "Population 2020", False)

# California land classfication color image
# Map.addLayer(composite, {'bands':['B4','B3','B2'], 'min': 0, 'max': 0.3},'California Color Image', 0);
# land classification legend
# Map.add_legend(legend_title="Land Cover Classification", legend_dict=legend_dict)
# //Add final map to the display
# Map.addLayer(finalmap.sldStyle(urbanPalette), {}, "Land Classification");

# Land classification
Map.addLayer(landcover, {}, 'NLCD Land Cover', False)
Map.add_legend(title="NLCD Land Cover Classification", legend_dict=legend_dict)

# Impervious land

Map.addLayer(imperviousMasked, {}, 'NLCD Impervious Land Cover', False)

# Impervious land scoring!
# Map.addLayer(imperviousZero, {'min': 0.0, 'max': 1.0, 'palette': ['white', 'black']}, 'ZERO NLCD Impervious Land Cover Masked', False)
# Map.addLayer(imperviousOne, {'min': 0.0, 'max': 1.0, 'palette': ['white', 'black']}, 'ONE NLCD Impervious Land Cover Masked', False)

# EVI land scoring!
# Map.addLayer(eviZero, {'min': 0.0, 'max': 4.0, 'palette': ['feedde','fdbe85','fd8d3c','d94701']}, 'ZERO EVI', False)
# Map.addLayer(eviOne, {'min': 0.0, 'max': 4.0, 'palette': ['feedde','fdbe85','fd8d3c','d94701']}, 'ONE EVI', False)
# Map.addLayer(eviTwo, {'min': 0.0, 'max': 4.0, 'palette': ['feedde','fdbe85','fd8d3c','d94701']}, 'TWO EVI', False)
# Map.addLayer(eviThree, {'min': 0.0, 'max': 4.0, 'palette': ['feedde','fdbe85','fd8d3c','d94701']}, 'THREE EVI', False)

# Land cover scoring!
# Map.addLayer(lcZero, {'min': 0.0, 'max': 3.0, 'palette': ['edf8b1','7fcdbb','2c7fb8']}, 'ZERO Land Cover', False)
# Map.addLayer(lcOne, {'min': 0.0, 'max': 3.0, 'palette': ['edf8b1','7fcdbb','2c7fb8']}, 'ONE Land Cover', False)
# Map.addLayer(lcTwo, {'min': 0.0, 'max': 3.0, 'palette': ['edf8b1','7fcdbb','2c7fb8']}, 'TWO Land Cover', False)

# Risk!
Map.addLayer(riskImg, risk_params, 'Risk 2020', True)

# Burning
Map.addLayer(burned2020, {'palette': ['black']}, 'Burned 2020', True)

roi = ee.Geometry.Point(-118.6304, 37.3003) # (long, lat)
Map.centerObject(roi, 6)
Map.addLayerControl()
Map